<a href="https://colab.research.google.com/github/Adityajl/3D-Image-Reconstruction-RANSAC-DinoV2/blob/master/3D_Image_Reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-matching-challenge-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71885%2F8143495%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240426%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240426T042250Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2cb9afae7477c250b639f86f81181fae247e3d2d5562a389dc7df44fdc49d9eefc53d6bfdb37fcfaf2f421169c273ce0eab53727954d016312a8f6e5bcfcbaf06c7283314b91398f8c19e666b2874668220cab0e3d21fe547deeb19d12277bcb9ae638294ce349eafad020b5684d1c81ea0bb5c060d43f649b4eeda822aa63d5137ed101d6ed17e527f278deee901623aab2e509de854bb4a9c4a023546c969d07e265634a1e683514c6cd7f03d3196d1f77858c379783ec52c6902e40cfd911488ba7cf58775fbc6a694ac3519366ac96895eac422569c478cd5067adc5662035d5c9f246af7dfd199e2aa10210655e2bd5bb7e09aa7df4cf2ea5c9ddc7755d,imc2024-packages-lightglue-rerun-kornia:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4628051%2F7884485%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240426%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240426T042250Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8eeb458d3f4da7b860b1f3928f0f4ef667ba2fcfdac86f5c1e53b3b6f5c21283de26fd68a43bcd3d5171f38e0767c380a892121b51604098313271ae879d3a5f1b47decd2e1859d444f7a6ca79aeda5210abfbaf11f4c3bbd6a936cd2e1dc75ae9baa3b8e76b8c202531bcd8a85120a96356bb360683ffb83e339d1630d9d6706e56bf64ffd9458d6a989eabe680aae9ba3e3611320233f6d25c286544c2ac7b40116ee3067e652c013fc05f87fa6a0d3ea3f14cbbb3ab766e4cf6a8918e75ed662496ec704de63ab70a5a462c01210cd46f4f83ffaed6e43209d01d4b416269bff497b94c40eb1289eed8e490050e3779a6cdb9cf6aff7baf290f060b379fec,colmap-db-import:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4628331%2F7884725%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240426%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240426T042250Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D611d98c49e22b490d6fde450bf99fe913c7027a53f2b233aac6ef73c1caf5c96d6da17153325d4c8bef8ddc5967c416c209aad72b8edcea84ad8a6c0f5a65a6632338cc8f646f4e508bdd959b8bb97e64fbbac8fb09ffed7fc9886c0ca2f5c0c558e1266e23c9bdc56f8c4bf18f8a584f852b430662232465b551b8c497088e5940fbc5ecfc478e9f92774d17adb5f61e88933f720949a177761ce0b99cae4662ae277e08906d4b4d67b5334a6590bffcd42b99f7a6d5af05ec199c72b8d151215a4646219ee07c00b113558054ac4ad09c034be9b18841a01016423f21551d211e8ee57ca6f7b273f448bbe175f1c02be6b8ee33c78b96ed34dcf1b92047174,dinov2/pytorch/base/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3326%2F4534%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240426%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240426T042250Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0eea18ac9466718930105ec76b05b9b32f4d6d3d6291bf1c2a322f79574d68f8a118bc4862fcd4939760a6fff5d6f1956f5636d5cec30c6ea7bdfa02fe269e4f7731edeb8cce517976491f800b2d34889ac9d017a6f6b647bae5a81459982c6ccdc201c6318466b235982f60f4d10ba92596ba3190bcc4671624cc51189a6d37c15fa8db51591727cea3816aebf6f60ce867e9f74681fbfa2c695c2e14d3eefef756bf27b97deeb1779d6fd70ce41663a309531b171c6a0237fd844fc3a3a641fcb9a428fd695737009e4fd67f1a89e139047f054d468168519f91ba6ec9f140376c788c983243f1bdbd145716a5883432e32e2aba183064b6520d6688471b16,lightglue/pytorch/aliked/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14317%2F17191%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240426%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240426T042250Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2a93a7694fedb341432d4135fcce551978b03b19bdde16e95f98d47e824cd89cf9d1f2eebe7210d6d9acbcfd6d355a50e81271aa288bff58b641e989d7d0f8c728df9d24c037045109904d9b88a717e9857f1c3fa400ff7f0f0c4c642b442a47f1b16154d3ac7b641f5651ccf924c44210caa35c2d4d3f274b30af37a7a8ef7bc8001c30c811ccae426ee3ee73832b3bcd4bace658fd0953e4193cae731cf9b0cafb0574c9d5dcbe176c825968f913872bd646dd9e7ef6e046d42ee56a40bea66192c2b39adbb06919a26b2804fabc0202f41cafcd045ba3711e31dfe6031cc65d8471a5fd709ec514b58c3d318a1439ea69c9aab15ae44337958441b978c1fe,aliked/pytorch/aliked-n16/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14611%2F17555%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240426%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240426T042250Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D14a4428140e2bb73ac4dfee4509f309f89266df3fa133f089d1c9a9f6f6090158ef0b19140cfe73832e4ffff94931913d0b0052a7285e6053ba295d75558d1dba8566069cd0dc1923a342c7695d1a3c67e9e7ee15bb7163672446a92d2623f0ed0b1cd4903bcd099b52fce16393491f429e272114e552c81ae65be2dba498cbd4f8cd0a7f10d32be499c4999e339a7b385bffd06d792bc2aa46aef34ffd2736c38d3d6906b1206ba23dc516e0ab8b76342c905cf05ed6b312bafbd3f21e4d3c82b5cce129cc10f8c820ec7cd2fa0060392dbcb28ea70457ccfc899c1ea30c5a4045c731134b564a0bda8ecf6c6c3ea3b1efa8ae3cb5cf341acddf39c280f54f1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 3722756243 bytes downloaded
Downloaded and uncompressed: image-matching-challenge-2024
[==================================================] 41275951 bytes downloaded
Downloaded and uncompressed: imc2024-packages-lightglue-rerun-kornia
[==================================================] 6000 bytes downloaded
Downloaded and uncompressed: colmap-db-import
[==================================================] 322024709 bytes downloaded
Downloaded and uncompressed: dinov2/pytorch/base/1
[==================================================] 44139405 bytes downloaded
Downloaded and uncompressed: lightglue/pytorch/aliked/1
[==================================================] 2507119 bytes downloaded
Downloaded and uncompressed: aliked/pytorch/aliked-n16/1
Data source import complete.


<h2>Reconstruct 3D scenes from 2D images over six different domains</h2>

In this notebook construct precise 3D maps using sets of images in diverse scenarios and environments.

Hope you enjoy ❤️

# Structure from Motion

Structure from Motion (SfM) is the name given to the procedure of **reconstructing a 3D scene and simultaneously obtaining the camera poses of a camera w.r.t. the given scene**. This means that, as the name suggests, we are creating the entire rigid structure from a set of images with different view points (or equivalently a camera in motion).

<center><img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.ENP48SmZHwG3r3O0lUVcWAHaFf%26pid%3DApi&f=1&ipt=550bf79efa85e7af870dd2d0a16793af7f3f83a36c14a1f4a648659a384b5a98&ipo=images" alt="Structure from motion structure: multiple cameras pointing toward an object in different positions and rotations that we need to find."></center>

# Baseline solution steps
In order to be able to estimate the camera poses, the solution provided by the organizers consists in the following steps:

* [1. Find pairs of images that are similar](#1)
* [2. Compute image keypoints](#2)
* [3. Match keypoints between images](#3)
* [4. Outlier detection with RANSAC](#4)
* [5. Sparse reconstruction](#5)

Let's understand how these steps are carried out

# Installing & importing relevant packages and models

In [2]:
!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth

Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia-0.7.2-py2.py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_moons-0.2.9-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_rs-0.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/rerun_sdk-0.15.0a2-cp38-abi3-manylinux_2_31_x86_64.whl


In [3]:
# General utilities
import matplotlib.pyplot as plt

import os
from tqdm import tqdm
from pathlib import Path
from time import time, sleep
from fastprogress import progress_bar
import gc
import numpy as np
import h5py
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from typing import Any
import itertools
import pandas as pd

# CV/MLe
import cv2
import torch
from torch import Tensor as T
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from PIL import Image
from transformers import AutoImageProcessor, AutoModel

import torch
from lightglue import match_pair
from lightglue import LightGlue, ALIKED
from lightglue.utils import load_image, rbd

# 3D reconstruction
import pycolmap

# Data importing into colmap
import sys
sys.path.append("/kaggle/input/colmap-db-import")

# Provided by organizers
from database import *
from h5_to_db import *

def arr_to_str(a):
    """Returns ;-separated string representing the input"""
    return ";".join([str(x) for x in a.reshape(-1)])

def load_torch_image(file_name: Path | str, device=torch.device("cpu")):
    """Loads an image and adds batch dimension"""
    img = K.io.load_image(file_name, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img

device = K.utils.get_cuda_device_if_available(0)
print(device)

DEBUG = len([p for p in Path("/kaggle/input/image-matching-challenge-2024/test/").iterdir() if p.is_dir()]) == 2
print("DEBUG:", DEBUG)

cuda:0
DEBUG: False


<a id="1"></a>
# Finding image pairs

To find pairs of similar images, we use [DINOv2](https://arxiv.org/pdf/2304.07193.pdf) to obtain normalized image embeddings.

<center><img src="https://www.labellerr.com/blog/content/images/2023/05/Dino-v2-20230419.jpg" alt="DINOv2 example"></center>
Then, we calculate the distances between all the embeddings, and only keep those below a given distance threshold. For images with less than a set minimum number of pairs, the closest ones are kept instead.

In [4]:
def embed_images(
    paths: list[Path],
    model_name: str,
    device: torch.device = torch.device("cpu"),
) -> T:
    """Computes image embeddings.

    Returns a tensor of shape [len(filenames), output_dim]
    """
    processor = AutoImageProcessor.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).eval().to(device)

    embeddings = []

    for i, path in tqdm(enumerate(paths), desc="Global descriptors"):
        image = load_torch_image(path)

        with torch.inference_mode():
            inputs = processor(images=image, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs) # last_hidden_state and pooled

            # Max pooling over all the hidden states but the first (starting token)
            # To obtain a tensor of shape [1, output_dim]
            # We normalize so that distances are computed in a better fashion later
            embedding = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=-1, p=2)

        embeddings.append(embedding.detach().cpu())
    return torch.cat(embeddings, dim=0)

In [5]:
def get_pairs_exhaustive(lst: list[Any]) -> list[tuple[int, int]]:
    """Obtains all possible index pairs of a list"""
    return list(itertools.combinations(range(len(lst)), 2))

def get_image_pairs(
    paths: list[Path],
    model_name: str,
    similarity_threshold: float = 0.6,
    tolerance: int = 1000,
    min_matches: int = 20,
    exhaustive_if_less: int = 20,
    p: float = 2.0,
    device: torch.device = torch.device("cpu"),
) -> list[tuple[int, int]]:
    """Obtains pairs of similar images"""
    if len(paths) <= exhaustive_if_less:
        return get_pairs_exhaustive(paths)

    matches = []

    # Embed images and compute distances for filtering
    embeddings = embed_images(paths, model_name)
    distances = torch.cdist(embeddings, embeddings, p=p)

    # Remove pairs above similarity threshold (if enough)
    mask = distances <= similarity_threshold
    image_indices = np.arange(len(paths))

    for current_image_index in range(len(paths)):
        mask_row = mask[current_image_index]
        indices_to_match = image_indices[mask_row]

        # We don't have enough matches below the threshold, we pick most similar ones
        if len(indices_to_match) < min_matches:
            indices_to_match = np.argsort(distances[current_image_index])[:min_matches]

        for other_image_index in indices_to_match:
            # Skip an image matching itself
            if other_image_index == current_image_index:
                continue

            # We need to check if we are below a certain distance tolerance
            # since for images that don't have enough matches, we picked
            # the most similar ones (which could all still be very different
            # to the image we are analyzing)
            if distances[current_image_index, other_image_index] < tolerance:
                # Add the pair in a sorted manner to avoid redundancy
                matches.append(tuple(sorted((current_image_index, other_image_index.item()))))

    return sorted(list(set(matches)))

In [6]:
if DEBUG:
    images_list = list(Path("/kaggle/input/image-matching-challenge-2024/test/church/images/").glob("*.png"))[:10]
    index_pairs = get_image_pairs(images_list, "/kaggle/input/dinov2/pytorch/base/1")
    print(index_pairs)

<a id="2"></a>
# Computing keypoints

In order to be able to know the position of each camera, we must be able to relate images to each other. For this, we extract relevant keypoints and compare pairs of image keypoints against each other. There are many ways to extract relevant keypoints, the most traditional one being [SIFT](https://en.wikipedia.org/wiki/Scale-invariant_feature_transform). However, newer and improved methods exist now, one of which is [ALIKED](https://arxiv.org/abs/2304.03608), the keypoint extraction method used in the solution.

<center><img src="https://www.catalyzex.com/_next/image?url=https%3A%2F%2Fd3i71xaburhd42.cloudfront.net%2Faf9fc17471b4c38211c3d9f5058c9c1f59501eea%2F3-Figure1-1.png&w=640&q=75" alt="ALIKED architecture"></center>


Let's take a closer look at the keypoints that ALIKED extracts.

In [7]:
if DEBUG:
    dtype = torch.float32 # ALIKED has issues with float16

    extractor = ALIKED(
            max_num_keypoints=4096,
            detection_threshold=0.01,
            resize=1024
        ).eval().to(device, dtype)

    path = images_list[0]
    image = load_torch_image(path, device=device).to(dtype)
    features = extractor.extract(image)

    fig, ax = plt.subplots(1, 2, figsize=(10, 20))
    ax[0].imshow(image[0, ...].permute(1,2,0).cpu())
    ax[1].imshow(image[0, ...].permute(1,2,0).cpu())
    ax[1].scatter(features["keypoints"][0, :, 0].cpu(), features["keypoints"][0, :, 1].cpu(), s=0.5, c="red")

    del extractor

In [8]:
def detect_keypoints(
    paths: list[Path],
    feature_dir: Path,
    num_features: int = 4096,
    resize_to: int = 1024,
    device: torch.device = torch.device("cpu"),
) -> None:
    """Detects the keypoints in a list of images with ALIKED

    Stores them in feature_dir/keypoints.h5 and feature_dir/descriptors.h5
    to be used later with LightGlue
    """
    dtype = torch.float32 # ALIKED has issues with float16

    extractor = ALIKED(
        max_num_keypoints=num_features,
        detection_threshold=0.01,
        resize=resize_to
    ).eval().to(device, dtype)

    feature_dir.mkdir(parents=True, exist_ok=True)

    with h5py.File(feature_dir / "keypoints.h5", mode="w") as f_keypoints, \
         h5py.File(feature_dir / "descriptors.h5", mode="w") as f_descriptors:

        for path in tqdm(paths, desc="Computing keypoints"):
            key = path.name

            with torch.inference_mode():
                image = load_torch_image(path, device=device).to(dtype)
                features = extractor.extract(image)

                f_keypoints[key] = features["keypoints"].squeeze().detach().cpu().numpy()
                f_descriptors[key] = features["descriptors"].squeeze().detach().cpu().numpy()

In [9]:
if DEBUG:
    feature_dir = Path("./sample_test_features")
    detect_keypoints(images_list, feature_dir)

<a id="3"></a>
# Match and compute keypoint distances

Now that we have the relevant image pairs and keypoints, we can go ahead and compare the keypoints of the images in a pair to find a good relationship between them. This is done with [LightGlue](https://arxiv.org/abs/2306.13643), which matches the keypoints and their descriptors between two images.

<center><img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fraw.githubusercontent.com%2Fcvg%2Flightglue%2Fmaster%2Fassets%2Feasy_hard.jpg&f=1&nofb=1&ipt=60962b56b05d3e8f95a064ab2a6010e5a6cbd5f1d10379d90e660b2561a3bae9&ipo=images" alt="LightGlue example"></center>

In [10]:
if DEBUG:
    matcher_params = {
        "width_confidence": -1,
        "depth_confidence": -1,
        "mp": True if 'cuda' in str(device) else False,
    }
    matcher = KF.LightGlueMatcher("aliked", matcher_params).eval().to(device)

    with h5py.File(feature_dir / "keypoints.h5", mode="r") as f_keypoints, \
         h5py.File(feature_dir / "descriptors.h5", mode="r") as f_descriptors:
            idx1, idx2 = index_pairs[0]
            key1, key2 = images_list[idx1].name, images_list[idx2].name

            keypoints1 = torch.from_numpy(f_keypoints[key1][...]).to(device)
            keypoints2 = torch.from_numpy(f_keypoints[key2][...]).to(device)
            print("Keypoints:", keypoints1.shape, keypoints2.shape)
            descriptors1 = torch.from_numpy(f_descriptors[key1][...]).to(device)
            descriptors2 = torch.from_numpy(f_descriptors[key2][...]).to(device)
            print("Descriptors:", descriptors1.shape, descriptors2.shape)

            with torch.inference_mode():
                distances, indices = matcher(
                    descriptors1,
                    descriptors2,
                    KF.laf_from_center_scale_ori(keypoints1[None]),
                    KF.laf_from_center_scale_ori(keypoints2[None]),
                )
    print(distances, indices)

In [11]:
def keypoint_distances(
    paths: list[Path],
    index_pairs: list[tuple[int, int]],
    feature_dir: Path,
    min_matches: int = 15,
    verbose: bool = True,
    device: torch.device = torch.device("cpu"),
) -> None:
    """Computes distances between keypoints of images.

    Stores output at feature_dir/matches.h5
    """

    matcher_params = {
        "width_confidence": -1,
        "depth_confidence": -1,
        "mp": True if 'cuda' in str(device) else False,
    }
    matcher = KF.LightGlueMatcher("aliked", matcher_params).eval().to(device)

    with h5py.File(feature_dir / "keypoints.h5", mode="r") as f_keypoints, \
         h5py.File(feature_dir / "descriptors.h5", mode="r") as f_descriptors, \
         h5py.File(feature_dir / "matches.h5", mode="w") as f_matches:

            for idx1, idx2 in tqdm(index_pairs, desc="Computing keypoing distances"):
                key1, key2 = paths[idx1].name, paths[idx2].name

                keypoints1 = torch.from_numpy(f_keypoints[key1][...]).to(device)
                keypoints2 = torch.from_numpy(f_keypoints[key2][...]).to(device)
                descriptors1 = torch.from_numpy(f_descriptors[key1][...]).to(device)
                descriptors2 = torch.from_numpy(f_descriptors[key2][...]).to(device)

                with torch.inference_mode():
                    distances, indices = matcher(
                        descriptors1,
                        descriptors2,
                        KF.laf_from_center_scale_ori(keypoints1[None]),
                        KF.laf_from_center_scale_ori(keypoints2[None]),
                    )

                # We have matches to consider
                n_matches = len(indices)
                if n_matches:
                    if verbose:
                        print(f"{key1}-{key2}: {n_matches} matches")
                    # Store the matches in the group of one image
                    if n_matches >= min_matches:
                        group  = f_matches.require_group(key1)
                        group.create_dataset(key2, data=indices.detach().cpu().numpy().reshape(-1, 2))

In [12]:
if DEBUG:
    keypoint_distances(images_list, index_pairs, feature_dir, verbose=False)

<a id="4"></a>
# RANSAC
Up to now, we have matched keypoints and their descriptors extracted from pairs of images. This is described by a [fundamental matrix](https://en.wikipedia.org/wiki/Fundamental_matrix_(computer_vision)) denoted as $F$. In epipolar geometry, with homogeneous image coordinates, $x$ and $x′$, of corresponding points in a stereo image pair, $Fx$ describes a line (an epipolar line) on which the corresponding point $x′$ on the other image must lie. That means, for all pairs of corresponding points, $x'Fx = 0$ holds. This is known as epipolar constraint or correspondance condition (or Longuet-Higgins equation), and is solved via the [eight-point algorithm](https://en.wikipedia.org/wiki/Eight-point_algorithm).

<center><img src="https://cmsc426.github.io/assets/sfm/epipole1.png" alt="Fundamental matrix"></center>

Since the keypoint correspondences are computed using feature descriptors, the data is bound to be noisy and (in general) contains several outliers. Thus, to remove these outliers, we use a [RANSAC](https://en.wikipedia.org/wiki/Random_sample_consensus) algorithm to find the best possible fundamental matrix. So, out of all possibilities, the $F$ matrix with maximum number of inliers is chosen.

<center><img src="https://cmsc426.github.io/assets/sfm/ransac.png" alt="RANSAC"></center>

In [13]:
def import_into_colmap(
    path: Path,
    feature_dir: Path,
    database_path: str = "colmap.db",
) -> None:
    """Adds keypoints into colmap"""
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, path, "", "simple-pinhole", single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )
    db.commit()

In [14]:
if DEBUG:
    database_path = "colmap.db"
    images_dir = images_list[0].parent
    import_into_colmap(
        images_dir,
        feature_dir,
        database_path,
    )

    # This does RANSAC
    pycolmap.match_exhaustive(database_path)

<a id="5"></a>
# Sparse Reconstruction

Now we have similar image pairs, with matched keypoint descriptors, without outliers! All that is left is to construct the scene and obtain the camera positions. We do this with pycolmap, which offers an incremental reconstruction algorithm that starts from two pairs of images and continually adds more and more images to the scene, resulting in a reconstructed scene with camera information. We can then use the camera rotation and translation as our submission!

In [15]:
if DEBUG:
    mapper_options = pycolmap.IncrementalPipelineOptions()
    mapper_options.min_model_size = 3
    mapper_options.max_num_models = 2

    maps = pycolmap.incremental_mapping(
        database_path=database_path,
        image_path=images_dir,
        output_path=Path.cwd() / "incremental_pipeline_outputs",
        options=mapper_options,
    )

In [16]:
if DEBUG:
    print(maps[0].summary())
    for k, im in maps[0].images.items():
        print("Rotation", im.cam_from_world.rotation.matrix(), "Translation:", im.cam_from_world.translation, sep="\n")
        print()

# Running everything

In [17]:
def parse_sample_submission(
    base_path: Path,
) -> dict[dict[str, list[Path]]]:
    """Construct a dict describing the test data as

    {"dataset": {"scene": [<image paths>]}}
    """
    data_dict = {}
    with open(base_path / "sample_submission.csv", "r") as f:
        for i, l in enumerate(f):
            # Skip header
            if i == 0:
                print("header:", l)

            if l and i > 0:
                image_path, dataset, scene, _, _ = l.strip().split(',')
                if dataset not in data_dict:
                    data_dict[dataset] = {}
                if scene not in data_dict[dataset]:
                    data_dict[dataset][scene] = []
                data_dict[dataset][scene].append(Path(base_path / image_path))

    for dataset in data_dict:
        for scene in data_dict[dataset]:
            print(f"{dataset} / {scene} -> {len(data_dict[dataset][scene])} images")

    return data_dict

In [18]:
def create_submission(
    results: dict,
    data_dict: dict[dict[str, list[Path]]],
    base_path: Path,
) -> None:
    """Prepares a submission file."""

    with open("submission.csv", "w") as f:
        f.write("image_path,dataset,scene,rotation_matrix,translation_vector\n")

        for dataset in data_dict:
            # Only write results for datasets with images that have results
            if dataset in results:
                res = results[dataset]
            else:
                res = {}

            # Same for scenes
            for scene in data_dict[dataset]:
                if scene in res:
                    scene_res = res[scene]
                else:
                    scene_res = {"R":{}, "t":{}}

                # Write the row with rotation and translation matrices
                for image in data_dict[dataset][scene]:
                    if image in scene_res:
                        print(image)
                        R = scene_res[image]["R"].reshape(-1)
                        T = scene_res[image]["t"].reshape(-1)
                    else:
                        R = np.eye(3).reshape(-1)
                        T = np.zeros((3))
                    image_path = str(image.relative_to(base_path))
                    f.write(f"{image_path},{dataset},{scene},{arr_to_str(R)},{arr_to_str(T)}\n")

In [19]:
class Config:
    base_path: Path = Path("/kaggle/input/image-matching-challenge-2024")
    feature_dir: Path = Path.cwd() / ".feature_outputs"

    device: torch.device = K.utils.get_cuda_device_if_available(0)

    pair_matching_args = {
        "model_name": "/kaggle/input/dinov2/pytorch/base/1",
        "similarity_threshold": 0.3,
        "tolerance": 500,
        "min_matches": 50,
        "exhaustive_if_less": 50,
        "p": 2.0,
    }

    keypoint_detection_args = {
        "num_features": 4096,
        "resize_to": 1024,
    }

    keypoint_distances_args = {
        "min_matches": 15,
        "verbose": False,
    }

    colmap_mapper_options = {
        "min_model_size": 3, # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
        "max_num_models": 2,
    }

In [20]:
def run_from_config(config: Config) -> None:
    results = {}

    data_dict = parse_sample_submission(config.base_path)
    datasets = list(data_dict.keys())

    for dataset in datasets:
        if dataset not in results:
            results[dataset] = {}

        for scene in data_dict[dataset]:
            images_dir = data_dict[dataset][scene][0].parent
            results[dataset][scene] = {}
            image_paths = data_dict[dataset][scene]
            print (f"Got {len(image_paths)} images")

            try:
                feature_dir = config.feature_dir / f"{dataset}_{scene}"
                feature_dir.mkdir(parents=True, exist_ok=True)
                database_path = feature_dir / "colmap.db"
                if database_path.exists():
                    database_path.unlink()

                # 1. Get the pairs of images that are somewhat similar
                index_pairs = get_image_pairs(
                    image_paths,
                    **config.pair_matching_args,
                    device=config.device,
                )
                gc.collect()

                # 2. Detect keypoints of all images
                detect_keypoints(
                    image_paths,
                    feature_dir,
                    **config.keypoint_detection_args,
                    device=device,
                )
                gc.collect()

                # 3. Match  keypoints of pairs of similar images
                keypoint_distances(
                    image_paths,
                    index_pairs,
                    feature_dir,
                    **config.keypoint_distances_args,
                    device=device,
                )
                gc.collect()

                sleep(1)

                # 4.1. Import keypoint distances of matches into colmap for RANSAC
                import_into_colmap(
                    images_dir,
                    feature_dir,
                    database_path,
                )

                output_path = feature_dir / "colmap_rec_aliked"
                output_path.mkdir(parents=True, exist_ok=True)

                # 4.2. Compute RANSAC (detect match outliers)
                # By doing it exhaustively we guarantee we will find the best possible configuration
                pycolmap.match_exhaustive(database_path)

                mapper_options = pycolmap.IncrementalPipelineOptions(**config.colmap_mapper_options)

                # 5.1 Incrementally start reconstructing the scene (sparse reconstruction)
                # The process starts from a random pair of images and is incrementally extended by
                # registering new images and triangulating new points.
                maps = pycolmap.incremental_mapping(
                    database_path=database_path,
                    image_path=images_dir,
                    output_path=output_path,
                    options=mapper_options,
                )

                print(maps)
                clear_output(wait=False)

                # 5.2. Look for the best reconstruction: The incremental mapping offered by
                # pycolmap attempts to reconstruct multiple models, we must pick the best one
                images_registered  = 0
                best_idx = None

                print ("Looking for the best reconstruction")

                if isinstance(maps, dict):
                    for idx1, rec in maps.items():
                        print(idx1, rec.summary())
                        try:
                            if len(rec.images) > images_registered:
                                images_registered = len(rec.images)
                                best_idx = idx1
                        except Exception:
                            continue

                # Parse the reconstruction object to get the rotation matrix and translation vector
                # obtained for each image in the reconstruction
                if best_idx is not None:
                    for k, im in maps[best_idx].images.items():
                        key = config.base_path / "test" / scene / "images" / im.name
                        results[dataset][scene][key] = {}
                        results[dataset][scene][key]["R"] = deepcopy(im.cam_from_world.rotation.matrix())
                        results[dataset][scene][key]["t"] = deepcopy(np.array(im.cam_from_world.translation))

                print(f"Registered: {dataset} / {scene} -> {len(results[dataset][scene])} images")
                print(f"Total: {dataset} / {scene} -> {len(data_dict[dataset][scene])} images")
                create_submission(results, data_dict, config.base_path)
                gc.collect()

            except Exception as e:
                print(e)

In [21]:
run_from_config(Config)

Looking for the best reconstruction
0 Reconstruction:
	num_reg_images = 40
	num_cameras = 40
	num_points3D = 12326
	num_observations = 64010
	mean_track_length = 5.19309
	mean_observations_per_image = 1600.25
	mean_reprojection_error = 0.908993
Registered: church / church -> 40 images
Total: church / church -> 41 images
/kaggle/input/image-matching-challenge-2024/test/church/images/00046.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00090.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00092.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00087.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00050.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00068.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00083.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00096.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00069.png
/kaggle/input/image-m